In [3]:
import os
from tqdm import tqdm
import numpy as np
import tiktoken
from datasets import load_dataset # huggingface datasets

# number of workers in .map() call
# good number to use is ~order number of cpu cores // 2
num_proc = 8
# takes 54GB in huggingface .cache dir, about 8M documents (8,013,769)
dataset = load_dataset("text", data_files={
    "train": '/home/jan/projects/LMStartup/data/cc_czech/nov_dec_50M/train.txt',
    "test": "/home/jan/projects/LMStartup/data/cc_czech/nov_dec_50M/test.txt"})
# owt by default only contains the 'train' split, so create a test split
dataset['val'] = dataset.pop('test') # rename the test split to val


Using custom data configuration default-466bc83e493ad028
Found cached dataset text (/home/jan/.cache/huggingface/datasets/text/default-466bc83e493ad028/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)
100%|██████████| 2/2 [00:00<00:00,  4.88it/s]


In [4]:
tokenizer.get_vocab_size()

NameError: name 'tokenizer' is not defined

In [ ]:
pre_tok = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit()]
)

In [ ]:
tokens = set()
for split in ['train', 'val']:
    for i in dataset[split]:

        words = list(map(lambda x:x[0],pre_tok.pre_tokenize_str(i['text'])))
        tokens.update(words)
#tokens

{'mínus',
 'čistotě',
 'dodání',
 'zapnutí',
 'drink',
 'spotřebitelské',
 'navštívíteli',
 'šapitó',
 'lidskosti',
 'vyuzit',
 'voňavých',
 'schůdné',
 'fíkovník',
 'prováděly',
 'proslulí',
 'jednání',
 '4praha',
 'tomato',
 'lakýrník',
 'prohřejí',
 'košutka',
 'letadly',
 'pmma',
 '1615',
 'četbě',
 'nuceným',
 'douglaska',
 'choco',
 'konglomerát',
 'distančním',
 'motivovala',
 '405ml',
 'sprchami',
 'návštěvníku',
 'střadatelé',
 'vtz',
 'rovnako',
 'pork',
 'všimlo',
 'defekace',
 'stava',
 'letky',
 'dečky',
 'rozhraním',
 'přestavěná',
 'zahaleny',
 'nezávislých',
 'kuchta',
 'odsouhlasili',
 'otočte',
 '2007',
 'učitelské',
 'osi',
 'ekonomičtí',
 'přerovského',
 'kódy',
 'kuřeti',
 'tržbami',
 'jílovitých',
 'přistávací',
 'projednáno',
 'doudera',
 'dne',
 'hříšníků',
 'lincolna',
 'blended',
 'jistý',
 'rozjet',
 'vodárenská',
 'stříbrná',
 'sporným',
 'přezdívaná',
 'strasti',
 'výstavu',
 'dlouhých',
 'nelichotivé',
 'nabídnuta',
 'zubov',
 'sen',
 'larev',
 'mitas',
 '

In [ ]:
vocab = {k: i for i, k in enumerate(tokens)}

In [ ]:
len(vocab)

176339

In [ ]:
sample_size = 4700000
def get_training_corpus():
    for i in range(0, len(dataset['train']), sample_size):
        yield dataset['train'][:]["text"]



In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

# create a tokenizer which splits on whitespace and punctuation

tokenizer = Tokenizer(models.WordLevel(vocab=vocab, unk_token="<unk>"))
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Lowercase(), normalizers.StripAccents()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)

special_tokens = ["<unk>", "<|endoftext|>"]
trainer = trainers.WordLevelTrainer( vocab_size=500000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

#tokenizer = Tokenizer.from_file("tokenizerBPE.json")


In [ ]:
tokenizer.get_vocab_size()

176330

In [ ]:
tokenizer = Tokenizer(models.BPE())

#tokenizer.pre_tokenizer = pre_tokenizers.Sequence(
#    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
#)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
tokenizer.decoder = decoders.ByteLevel()
trainer = trainers.BpeTrainer(vocab_size=50000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
tokenizer.save("cc/nov_dec_2022/tokenizerWL.json")

In [ ]:
tt=pre_tokenizers.ByteLevel()
len(tt.alphabet())

256

In [ ]:
snt = tokenizer.encode('this is really great stuff.')
snt.ids

[49, 21, 865, 612, 3613, 15]

In [ ]:
import pickle
with open('cc/nov_dec_2022/tokenized.pkl', 'rb') as f:
    tokenized = pickle.load(f)

/home/jan/miniconda3/envs/torch114/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer.get_vocab_size()

10001

In [6]:
2**(16)

65536

In [8]:
import numpy as np
import os
from tqdm import tqdm

#!!!!!!!!!!!!!! change dtype to np.uint16 if vocab size < 2**16

# concatenate all the ids in each dataset into one large file we can use for training
for split, dset in tokenized.items():
    arr_len = np.sum(dset['len'])
    filename = os.path.join(os.path.dirname('cc/nov_dec_2022'), f'{split}.bin')
    dtype = np.uint32 # (can do since enc.max_token_value == 50256 is < 2**16)
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))

    print(f"writing {filename}...")
    idx = 0
    for example in tqdm(dset):
        arr[idx : idx + example['len']] = example['ids']
        idx += example['len']
    arr.flush()

# train.bin is ~17GB, val.bin ~8.5MB
# train has ~9B tokens (9,035,582,198)
# val has ~4M tokens (4,434,897)

# to read the bin files later, e.g. with numpy:
# m = np.memmap('train.bin', dtype=np.uint16, mode='r')

writing cc/train.bin...


100%|██████████| 1175284/1175284 [01:09<00:00, 16843.07it/s]


writing cc/val.bin...


100%|██████████| 11872/11872 [00:00<00:00, 15150.99it/s]


In [11]:
9000000000/40000000

225.0

In [12]:
225*200

45000

In [ ]:
import numpy as np
m = np.memmap('train.bin', dtype=np.int16, mode='r')

In [ ]:
import tiktoken
enc = tokenizer
lst = tokenized['train'][0]['ids']
for l in lst:
    print(enc.decode([l]))
enc.decode(lst)

staÄįÃŃ
Ġi
ĠtroÅ¡ku
ĠdÃ©le
Ġpodr
Å¾et
Ġikonu
Ġaplikace
Ġmy
Ġair
Ġna
ĠploÅ¡e
Ġtelefonu
Ġa
ĠotevÅĻe
Ġse
ĠvÃ¡m
ĠrychlÃ©
Ġmenu
Ġspolu
Ġs
Ġtouto
ĠmoÅ¾nostÃŃ



'staÄįÃŃ Ġi ĠtroÅ¡ku ĠdÃ©le Ġpodr Å¾et Ġikonu Ġaplikace Ġmy Ġair Ġna ĠploÅ¡e Ġtelefonu Ġa ĠotevÅĻe Ġse ĠvÃ¡m ĠrychlÃ© Ġmenu Ġspolu Ġs Ġtouto ĠmoÅ¾nostÃŃ'

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
enc.encode('this is test<|endoftext|>',allowed_special={'<|endoftext|>'})

[5661, 318, 1332, 50256]

In [ ]:
dir(enc)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_core_bpe',
 '_encode_bytes',
 '_encode_only_native_bpe',
 '_encode_single_piece',
 '_mergeable_ranks',
 '_pat_str',
 '_special_tokens',
 'decode',
 'decode_bytes',
 'decode_single_token_bytes',
 'decode_tokens_bytes',
 'encode',
 'encode_batch',
 'encode_ordinary',
 'encode_ordinary_batch',
 'encode_single_token',
 'encode_with_unstable',
 'eot_token',
 'max_token_value',
 'n_vocab',
 'name',
 'special_tokens_set',
 'token_byte_values']

In [ ]:
enc.n_vocab

50257